This notebook loads a dataset and trains a gradient boosting regressor to use to predict the thresholds.  The notebook plays a UCB and the vegas pull 2 bot.  It is not a very good bot, but with some work could be.  Commit the note and submit the submit.tar.gz file - it has the notebook and the model in the file.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('../input/santa-train/train_santa.csv')

#df = pd.read_csv('../input/santa-train/train_santa-1.csv')

#df=df.sample(frac=0.15, replace=False, random_state=1)

#df = df[df['pulls'] > 2]

y = df['target'].values

#df['step'] = df['step'].multiply(.0001)

X = df.drop(['target','step','opp_pull'],axis=1).values

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.05,random_state=42)

df

In [ ]:
#Setup a regressor
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'auc'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000,
    "n_estimators": 1000
}

#reg = lgb.LGBMRegressor()

#model=reg.fit(X_train, y_train)


#reg = RandomForestRegressor()
reg = GradientBoostingRegressor()
model=reg.fit(X_train,y_train)

pred = model.predict(X_test)
#Cost Function
mse = mean_squared_error(y_test,pred)
print (reg.score(X_test,y_test))
print (mse)

In [ ]:
!pip install kaggle-environments --upgrade

from kaggle_environments import make

!mkdir /kaggle_simulations
!mkdir /kaggle_simulations/agent
!mkdir /kaggle_simulations/agent/saved_model

In [ ]:
# save the model to disk
import pickle
filename = '/kaggle_simulations/agent/saved_model/model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
%%writefile /kaggle_simulations/agent/main.py

import math
from scipy.stats import beta
import numpy as np
import random
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# load the model from disk
import pickle
filename = '/kaggle_simulations/agent/saved_model/model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

total_reward = 0

def ucb_agent(observation, configuration):    
    global wins, pulls, opp_pull, last_bandit, total_reward, opp_bandit
    
    if observation.step == 0:
        wins = [0] * configuration["banditCount"]
        
        pulls = [0] * configuration["banditCount"]
        
        opp_pull = [0] * configuration["banditCount"]
        
        opp_bandit=[]
        
        chosen_bandit = random.randint(0,99)
        last_bandit = chosen_bandit
        
        return int(chosen_bandit)
    
    if observation.step > 0:
        
        player = int(last_bandit == observation.lastActions[1])
        
        if player==0:
            opponent=1
        else:
            opponent=0
        
        reward = observation.reward - total_reward
        total_reward += reward
        
        if reward == 1:
            wins[last_bandit] += 1
        pulls[last_bandit] += 1
        opp_pull[observation['lastActions'][opponent]] += 1
        
        #if observation.step < 500 and reward ==1:
            #return last_bandit
        
        opp_bandit.append(observation['lastActions'][opponent])
        #'''
        if len(opp_bandit)>15 and observation.step < 1700:
            length=len(opp_bandit)
            if opp_bandit[length-1] == opp_bandit[length-2] and opp_bandit[length-1] != opp_bandit[length-15]:
                last_bandit=opp_bandit[length-1]
                return opp_bandit[length-1]
        #'''
        best_proba = -1
        best_agent = None
        dat=[]
        for k in range(configuration["banditCount"]):
            to_append = wins[k],pulls[k]#,opp_pull[k],observation.step
            dat.append(to_append)
            
        proba = loaded_model.predict(dat)
        
        chosen_bandit = int(np.argmax(proba))
        #'''
        
        if observation.step <= 800:
            N = 5
            res = sorted(range(len(proba)), key = lambda sub: proba[sub])[-N:]
            chosen_bandit=random.choice(res)
            
        #''' 
        last_bandit = int(chosen_bandit)
        return int(chosen_bandit)

In [ ]:
%%writefile bayesian_ucb.py

import numpy as np
from scipy.stats import beta

post_a, post_b, bandit = [None] * 3
total_reward = 0
c = 3

def agent(observation, configuration):
    global total_reward, bandit, post_a, post_b, c

    if observation.step == 0:
        post_a, post_b = np.ones((2, configuration.banditCount))
    else:
        r = (observation.reward - total_reward)
        total_reward = observation.reward
        # Update Gaussian posterior
        post_a[bandit] += r
        post_b[bandit] += 1 - r
    
    bound = post_a / (post_a + post_b) + beta.std(post_a, post_b) * c
    bandit = int(np.argmax(bound))
    
    return bandit

In [ ]:
%%writefile vegas_pull2.py

import numpy as np
import pandas as pd
import random, os, datetime, math
from collections import defaultdict

total_reward = 0
bandit_dict = {}

def set_seed(my_seed=42):
    os.environ['PYTHONHASHSEED'] = str(my_seed)
    random.seed(my_seed)
    np.random.seed(my_seed)

def get_next_bandit():
    best_bandit = 0
    best_bandit_expected = 0
    for bnd in bandit_dict:
        expect = (bandit_dict[bnd]['win'] - bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'] - (bandit_dict[bnd]['opp']>0)*1.5 + bandit_dict[bnd]['op_continue']) \
                 / (bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp']) \
                * math.pow(0.97, bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'])
        if expect > best_bandit_expected:
            best_bandit_expected = expect
            best_bandit = bnd
    return best_bandit

my_action_list = []
op_action_list = []

op_continue_cnt_dict = defaultdict(int)

def multi_armed_probabilities(observation, configuration):
    global total_reward, bandit_dict

    my_pull = random.randrange(configuration['banditCount'])
    if 0 == observation['step']:
        set_seed()
        total_reward = 0
        bandit_dict = {}
        for i in range(configuration['banditCount']):
            bandit_dict[i] = {'win': 1, 'loss': 0, 'opp': 0, 'my_continue': 0, 'op_continue': 0}
    else:
        last_reward = observation['reward'] - total_reward
        total_reward = observation['reward']
        
        my_idx = observation['agentIndex']
        my_last_action = observation['lastActions'][my_idx]
        op_last_action = observation['lastActions'][1-my_idx]
        
        my_action_list.append(my_last_action)
        op_action_list.append(op_last_action)
        
        if 0 < last_reward:
            bandit_dict[my_last_action]['win'] = bandit_dict[my_last_action]['win'] +1
        else:
            bandit_dict[my_last_action]['loss'] = bandit_dict[my_last_action]['loss'] +1
        bandit_dict[op_last_action]['opp'] = bandit_dict[op_last_action]['opp'] +1
        
        if observation['step'] >= 3:
            if my_action_list[-1] == my_action_list[-2]:
                bandit_dict[my_last_action]['my_continue'] += 1
            else:
                bandit_dict[my_last_action]['my_continue'] = 0
            if op_action_list[-1] == op_action_list[-2]:
                bandit_dict[op_last_action]['op_continue'] += 1
            else:
                bandit_dict[op_last_action]['op_continue'] = 0
        
        if last_reward > 0:
            my_pull = my_last_action
        else:
            if observation['step'] >= 4:
                if (my_action_list[-1] == my_action_list[-2]) and (my_action_list[-1] == my_action_list[-3]):
                    if random.random() < 0.5:
                        my_pull = my_action_list[-1]
                    else:
                        my_pull = get_next_bandit()
                else:
                    my_pull = get_next_bandit()
            else:
                my_pull = get_next_bandit()
    
    return my_pull

In [ ]:
def print_rounds(file1, file2, N=3):
    env = make("mab", debug=True)
    p1_count=0
    p2_count=0
    print ('simulating...',N,'games')
    for i in range(N):
        game=env.run([file1, file2])
        p1_score = env.steps[-1][0]['reward']
        p2_score = env.steps[-1][1]['reward']
        if p1_score>p2_score:
            p1_count+=1
        else:
            p2_count+=1
        env.reset()
        z=i+1
        #print(f"Round {i+1}: {p1_score} - {p2_score}")
    print (p1_count,'for',z,round(p1_count/z,3),'.vs',round(p2_count/z,3))
    print ('complete')
    points_est1=[]
    points_est2=[]
    
    for x in range(2000):
        #print (game[x][1]['reward'])
        z=x+1
        points_est1.append(game[x][0]['reward']/z)
        points_est2.append(game[x][1]['reward']/z)
        
    plt.plot(points_est1,label='test_bot')
    plt.plot(points_est2, label='baseline')
    plt.legend()
    plt.show()
    print (sum(points_est2)/len(points_est2))
    
    #n=500
    #chunked=[points_est2[i:i + n] for i in range(0, len(points_est2), n)]
    #for i in range(len(chunked)):
        #print (sum(chunked[i])/len(chunked[i]))
        #print(len(chunked[i]))
        

In [ ]:
env = make("mab", debug=True)
env.run(["/kaggle_simulations/agent/main.py", "bayesian_ucb.py"])
env.render(mode="ipython", width=800, height=400)

In [ ]:
env = make("mab", debug=True)
env.run(["/kaggle_simulations/agent/main.py", "vegas_pull2.py"])
env.render(mode="ipython", width=800, height=400)

In [ ]:
print('test bot vs bayesian_ucb')
print_rounds("/kaggle_simulations/agent/main.py", "bayesian_ucb.py")

In [ ]:
print('test bot vs vegas_pull2')
print_rounds("/kaggle_simulations/agent/main.py", "vegas_pull2.py")

In [ ]:
!cd /kaggle_simulations/agent && tar -czvf /kaggle/working/submit.tar.gz main.py saved_model